In [127]:
#import libraries
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.model_selection import train_test_split



In [128]:
#loading subjects data
data_s = pd.read_csv("dbworld_subjects_stemmed.csv")

In [129]:
#separate features and class
xb = data_s.iloc[:,1:-1]
yb = data_s['CLASS']

# yb.head()

In [130]:
#Splitting data for training and testing
xb_train, xb_test, yb_train, yb_test = train_test_split(xb,yb,test_size = 0.2,random_state = 10)

# xb_train.head()

In [131]:
#Concatenate train features with train class
trainb = pd.concat([xb_train, yb_train], axis=1)
trainb.head()

,10th,13th,1st,2nd,31st,3rd,5th,6th,abstract,academ,...,vacanc,valencia,vehicular,video,web,wireless,workflow,workshop,zurich,CLASS
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [132]:
#separate spam and ham msgs
classgrpb = trainb.groupby('CLASS')
spam_bf = classgrpb.get_group(1)
ham_bf = classgrpb.get_group(0)
spam_bf.head()


,10th,13th,1st,2nd,31st,3rd,5th,6th,abstract,academ,...,vacanc,valencia,vehicular,video,web,wireless,workflow,workshop,zurich,CLASS
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
47,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
43,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
31,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [133]:
#removing class column at the end 
spam_b = spam_bf.iloc[:,:-1]
ham_b = ham_bf.iloc[:,:-1]

In [134]:
#probability of spam and ham messages
p_spamb = len(spam_b) / len(trainb)
p_hamb = len(ham_b) / len(trainb)
print(len(spam_b), len(trainb))

25 51


In [135]:
#counting words in spam and ham of each mail
nspamb_rows = spam_b.sum(axis=1)
nspamb_rows.head()

nhamb_rows = ham_b.sum(axis=1)
nhamb_rows.head()



21    4
7     8
60    9
50    6
10    9
dtype: int64

In [136]:
#counting words of spam and ham of all mails
nspamb = nspamb_rows.sum(axis=0)
nhamb = nhamb_rows.sum(axis=0)



In [137]:
#laplace parameter and number of classes
k = 1
vocab = list(xb_train.columns)
# N_vocab = xb_train.shape[1]
n_vocab = len(vocab)


In [138]:
# Calculate probabilities of words given spam or ham
prob_word_given_spam = {word:0 for word in vocab}
prob_word_given_ham = {word:0 for word in vocab}


for word in vocab:
   nword_given_spam = spam_b[word].sum()
   pword_given_spam = (nword_given_spam + k) / (nspamb + (k*n_vocab))
   prob_word_given_spam[word] = pword_given_spam

   nword_given_ham = ham_b[word].sum() 
   pword_given_ham = (nword_given_ham + k) / (nhamb + (k*n_vocab))
   prob_word_given_ham[word] = pword_given_ham


In [139]:
#prediction function
def predict(test):

    prob_spam_given_mail = np.log(p_spamb)
    prob_ham_given_mail = np.log(p_hamb)
    # print(prob_spam_given_mail,prob_spam_given_mail)
    test = test.reset_index()
    test = test.iloc[:,1:]
    # print(test)
    l = []
    for i in range(test.shape[0]):
        # for word in vocab:
        for word in vocab:
            if test.loc[i][word] == 1:
                prob_spam_given_mail += np.log(prob_word_given_spam[word])
                prob_ham_given_mail += np.log(prob_word_given_ham[word])

            
        # print(prob_ham_given_mail,prob_ham_given_mail)
        if prob_spam_given_mail > prob_ham_given_mail:
            l.append(1)
        elif prob_ham_given_mail > prob_spam_given_mail:
            l.append(0)
        else:
            l.append(0.5)
    
    return l


In [140]:
#Predicting test data
yb_test_pred = predict(xb_test)


In [141]:
#Calculating f measure
def fmeasure(pred_values, true_values):
	
	TP, TN, FP, FN = 0.0, 0.0, 0.0, 0.0

	for p, a in zip(pred_values, true_values):
		if p == 0 and a == 0:
			TP = TP + 1
		elif p == 1 and a == 1:
			TN = TN + 1
		elif p == 0 and a == 1:
			FP = FP + 1
		elif p == 1 and a == 0:
			FN = FN + 1

	Pre = TP / (TP + FP)
	Rec = TP / (TP + FN)

	f_measure = (2 * Pre * Rec) / (Pre + Rec)

	return f_measure

In [142]:
#Printing f measure
f_measure_subjects = fmeasure(yb_test,yb_test_pred)
print("f_measure of email subject: ",f_measure_subjects)


f_measure of email subject:  0.8181818181818181


Email Subject provides better classification since it has more f measure value compared to email body